# Deskripsi
Deskripsi mengenai variable-variabel yang ada dalam penelitian ini akan dijelaskan dalam beberapa bagian:

1. Labeling
2. Loading
3. 'News' Variables Description

### Karakteristik Ekonomi 
##### (Eurostat Business Register,2010)
1. Klasifikasi Aktivitas Ekonomi (KBLI)
    * Aktivitas ekonomi berdasarkan bahan produksi
2. Nilai tambah Perusahaan
3. Ukuran-ukuran Perusahaan
    * Jumlah pekerja
    * *Turnover* (Nilai ambil alih)
4. Ukuran-ukuran Perusahaan berdasarkan Balance sheet
    * Aset
    * Inventori
    * Cash
    * Dividen
    * Obligasi
5. Ukuran Perusahaan berdasarkan Upah dan Gaji
    * Upah (*Wage*)
    * Gaji (*Salary*)

### Karakteristik Demografi
##### (Eurostat Business Register,2010)
1. Active and Restart
    * Birth
    * Death
    * Reactivation
    * Real Death and Real Birth
2. Survive
    * Break-Up
    * Split-Off
    * Merger
    * Takeover
3. Structure
    * Change of Ownership
    * Change of Group
    * Restructure within enterprise/enterprise group
    * Complex Restructure

### Karakteristik Link dan Acuan Eksternal
##### (Pedoman Statistical Business Registers,2015)
1. Persentase Kepemilikan
2. Hubungan antar 2 perusahaan
    * Anak perusahaan

In [1]:
import matplotlib.pyplot as plt
import os, re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from datetime import datetime as dt
import cPickle as pickle
import re #Regex
import seaborn as sns
import pandas as pd
sns.set()
%matplotlib inline

In [2]:
DATAStem    = pd.DataFrame()
DATAPost    = pd.DataFrame()
DATAPost_sk = pd.DataFrame()

#Load from Striped Text
%time DATAPost['Judul'] = pickle.load( open( "v1.4\DATA_JUDULStriped-v4a.p", "rb" ) )
%time DATAPost['Short'] = pickle.load( open( "v1.4\DATA_SHORTStriped-v4a.p", "rb" ) )
%time DATAPost['Long']  = pickle.load( open( "v1.4\DATA_LONGStriped-v4a.p", "rb" ) )

#Load from Striped Text
%time DATAPost_sk['Judul'] = pickle.load( open( "v1.4\DATA_JUDULStriped-v4c.p", "rb" ) )
%time DATAPost_sk['Short'] = pickle.load( open( "v1.4\DATA_SHORTStriped-v4c.p", "rb" ) )
%time DATAPost_sk['Long']  = pickle.load( open( "v1.4\DATA_LONGStriped-v4c.p", "rb" ) )

%time DATA_TARGET    = pickle.load( open( "v1.4\DATA_TARGET-v4a.p", "rb" ) )
%time DATA_TARGET_sk = pickle.load( open( "v1.4\DATA_TARGET-v4c.p", "rb" ) )

# DATA_TARGET = DATA_TARGET.sort_index()
# DATAPost = DATAPost.sort_index()

DATA_TARGET    = DATA_TARGET.reset_index(drop=True)
DATA_TARGET_sk = DATA_TARGET_sk.reset_index(drop=True)
DATAPost    = DATAPost.reset_index(drop=True)
DATAPost_sk = DATAPost_sk.reset_index(drop=True)

Wall time: 9 ms
Wall time: 18 ms
Wall time: 150 ms
Wall time: 3 ms
Wall time: 3 ms
Wall time: 25 ms
Wall time: 7 ms
Wall time: 1e+03 µs


In [3]:
%time postagger = pickle.load(open( "POSTAGGER.p", "rb" ))
%time tagList   = pickle.load(open( "TAGLIST.p", "rb" ))

import string
import operator

def toNVFraction(data):
    def combine_dicts(a, b, op=operator.add):
        return dict(a.items() + b.items() +
            [(k, op(a[k], b[k])) for k in set(b) & set(a)])
    
    def onlyAZ(teks):
        return re.sub(r'[^a-zA-Z]', ' ', teks)
    
    def onlyNVFractionSentence(teks):
        splitted = postagger.tag(onlyAZ(teks).split())
        N  = len(splitted)
        word = 0
        k = {}
        for i in tagList:k[i] = float (0)
        for word,pos in splitted:
            k[pos]+=1
        return N,k

    def onlyNVFractionParagraph(par):
        splittedPar = par.split('.')
        word = 0
        _k = {}
        for i in tagList:_k[i] = float (0)
        for sentence in splittedPar:
            w,k = onlyNVFractionSentence(sentence)
            word+=w
            _k = {x: _k.get(x, 0) + k.get(x, 0) for x in set(_k)}
        #return noun/word,verb/word
        return _k.values()

    
    balikan = np.array([0 for i in range(0,len(tagList))])
    for teks in data:
        frac = onlyNVFractionParagraph(teks)
        balikan = np.vstack((balikan,frac))
    return balikan[1:]

Wall time: 301 ms
Wall time: 0 ns


In [4]:
%time JUDULFrac = toNVFraction(DATAPost['Judul'])
%time SHORTFrac = toNVFraction(DATAPost['Short'])
%time LONGFrac  = toNVFraction(DATAPost['Long'])

%time JUDULFrac_sk = toNVFraction(DATAPost_sk['Judul'])
%time SHORTFrac_sk = toNVFraction(DATAPost_sk['Short'])
%time LONGFrac_sk  = toNVFraction(DATAPost_sk['Long'])

Wall time: 216 ms
Wall time: 266 ms
Wall time: 3.37 s
Wall time: 32 ms
Wall time: 32 ms
Wall time: 233 ms


In [5]:
cek = DATAPost.head()
cek

,Judul,Short,Long
0,6 keuntungan mengambil kpr saat masih lajang,saat masih lajang memang kebutuhan tak terlalu...,merdeka.com - memiliki rumah menjadi impian se...
1,pria ini tega ajak anaknya bersepatu roda seja...,jarak itu ditempuh selama 14 hari.,merdeka.com - hidup memang tidak mudah. seoran...
2,harga emas antam dibuka stagnan di rp 566.000 ...,harga pembelian emas dan harga penjualan kemba...,merdeka.com - harga emas batangan di unit bisn...
3,jagonya ayam gemilang di silverstone,ketiganya kembali meraih peringkat keempat dal...,merdeka.com - para pebalap jagonya ayam tampil...
4,pebalap jagonya ayam finish di posisi lima,trio pebalap jagonya ayam yang bernaung dengan...,merdeka.com - meski terkendala dengan masalah ...


In [6]:
for i in cek['Judul']:
    print postagger.tag(re.sub(r'[^a-zA-Z]', ' ', i).split())

[('keuntungan', 'NN'), ('mengambil', 'NN'), ('kpr', 'NN'), ('saat', 'SC'), ('masih', 'RB'), ('lajang', 'NN')]
[('pria', 'NN'), ('ini', 'DT'), ('tega', 'NN'), ('ajak', 'NN'), ('anaknya', 'NN'), ('bersepatu', 'NN'), ('roda', 'NN'), ('sejauh', 'NN'), ('kilometer', 'NN')]
[('harga', 'NN'), ('emas', 'NN'), ('antam', 'NN'), ('dibuka', 'VB'), ('stagnan', 'NN'), ('di', 'IN'), ('rp', 'NN'), ('per', 'NN'), ('gram', 'NN')]
[('jagonya', 'NN'), ('ayam', 'NN'), ('gemilang', 'NN'), ('di', 'IN'), ('silverstone', 'NN')]
[('pebalap', 'NN'), ('jagonya', 'NN'), ('ayam', 'NN'), ('finish', 'NN'), ('di', 'IN'), ('posisi', 'NN'), ('lima', 'CD')]


In [7]:
DATAPost['Short'] = pickle.load( open( "v1.4\DATA_SHORTStriped-v2.p", "rb" ) )
DATA_SHORTPos = DATAPost['Short']
k = DATA_SHORTPos[DATA_SHORTPos.index == 731].values
print toNVFraction(k)
print tagList

[[ 0.  0.  1.  0.  1.  0.  0.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
['FW', 'NEG', 'JJ', 'WP', 'DT', 'PENGOLAHAN', 'RP', 'NN', 'PRL', 'PRN', '-', ',', '.', 'PRP', 'RB', ':', 'NNS', 'NNP', 'VB', 'WRB', 'CC', 'CD', 'IN', 'MD', '--', 'SYM', None, 'UH', 'SC']


In [8]:
pickle.dump( JUDULFrac, open( "v2.0\DATA_JUDUL-v6a.p", "wb" ) )
pickle.dump( SHORTFrac, open( "v2.0\DATA_SHORT-v6a.p", "wb" ) )
pickle.dump( LONGFrac,  open( "v2.0\DATA_LONG-v6a.p", "wb" ) )
pickle.dump( DATA_TARGET,  open( "v2.0\DATA_TARGET-v6a.p", "wb" ) )

pickle.dump( JUDULFrac_sk, open( "v2.0\DATA_JUDUL-v6c.p", "wb" ) )
pickle.dump( SHORTFrac_sk, open( "v2.0\DATA_SHORT-v6c.p", "wb" ) )
pickle.dump( LONGFrac_sk,  open( "v2.0\DATA_LONG-v6c.p", "wb" ) )
pickle.dump( DATA_TARGET_sk,  open( "v2.0\DATA_TARGET-v6c.p", "wb" ) )